In [1]:
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

--2022-01-23 16:23:44--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.53.2
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.53.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58176133 (55M) [text/tab-separated-values]
Saving to: ‘quora_duplicate_questions.tsv’

quora_duplicate_que 100%[===================>]  55.48M  3.17MB/s    in 19s     

2022-01-23 16:24:03 (2.94 MB/s) - ‘quora_duplicate_questions.tsv’ saved [58176133/58176133]



In [2]:
from random import seed, sample, shuffle
from utils import load_dataset, gather_text, TextVectorizer


def get_quora_data():
    data = open('quora_duplicate_questions.tsv', 'r').readlines()
    corpus = []

    # we do not want the header to be included 
    for line in data[1:]:
        line = line.split('\t')
        try:
            # If this cannot be done, there is a problem and we do not want to save this example
            text_a, text_b, label = line[-3], line[-2], line[-1].strip()
            int(label) # just a test, to make sure that the label is convertible to int
            if len(text_a) != 0 and len(text_b) != 0: # make sure that the text is not empty
                corpus.append([text_a, text_b, label])
        except:
            pass
    
    matched = [c for c in corpus if c[-1] == "1"]
    unmatched = [c for c in corpus if c[-1] == "0"]

    seed(32)
    part1 = matched
    part2 = sample(unmatched, 149263)

    train = part1[:130000] + part2[:130000]
    dev = part1[130000:140000] + part2[130000:140000]
    test = part1[140000:] + part2[140000:]

    shuffle(train)
    shuffle(dev)
    shuffle(test)

    def save(dataset, fpath):
        dataset = ['\t'.join(d) for d in dataset]
        with open(fpath, 'w') as f:
            f.write('\n'.join(dataset))
            f.close()
            print(f"{fpath} has been saved!")

    save(train, "train.txt")
    save(dev, "dev.txt")
    save(test, "test.txt")
    

def build_dict():
    train_set = load_dataset('train.txt')
    text = gather_text(train_set)
    V = TextVectorizer()
    V.build_vocab(text)
    V.save_vocab_as_json()

In [3]:
get_quora_data()
build_dict()

train.txt has been saved!
dev.txt has been saved!
test.txt has been saved!
Two vocabulary dictionaries have been built!
Please call X.vocab_to_idx | X.idx_to_vocab to find out more where [X] stands for the name you used for this TextVectorizer class.
vocab_to_idx.json has been successfully saved!
